### Import Baseball Savant Data and Remove Position Players who pitched. Also add new features for 2 out walks, 0-2/1-2 walks, and leadoff walks.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
import sys

# reference main directory in existing folder
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import matplotlib.pyplot as plt
from src.paths import RAW_DATA_DIR
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

# show all rows and columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# import data
savant = pd.read_csv(RAW_DATA_DIR / 'savant_pitch_level.csv')
data_dictionary = pd.read_excel(RAW_DATA_DIR / 'data_dictionary.xlsx')
fangraph = pd.read_csv(RAW_DATA_DIR / 'fangraphs_season_level.csv')

# External Data
baseball_reference = pd.read_excel(RAW_DATA_DIR / 'sportsref_download.xlsx')

In [2]:
baseball_reference.shape

(889, 9)

### 1. Import Baseball Reference Data to find all position players who pitched in from 2019-2023

In [3]:
# exlude shohei ohtani from data
baseball_reference = baseball_reference[baseball_reference['Name'] != 'Shohei Ohtani']

In [4]:
import unicodedata

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return "".join([c for c in nfkd_form if not unicodedata.combining(c)])

baseball_reference['Name'] = baseball_reference['Name'].apply(remove_accents)
savant.player_name = savant.player_name.apply(remove_accents)
fangraph.Name = fangraph.Name.apply(remove_accents)



In [5]:
baseball_reference_names = baseball_reference['Name']
# get a list of names from baseball reference
baseball_reference_names = baseball_reference['Name'].tolist()
savant_names = savant['player_name']
fangraph_names = fangraph['Name']
# split the player_name into last name and first name from the comma and space
savant['Last Name'] = savant['player_name'].str.split(', ').str[0]
savant['First Name'] = savant['player_name'].str.split(', ').str[1]
savant['Name'] = savant['First Name'] + ' ' + savant['Last Name']
savant['player_name'] = savant['Name']
#drop the Name column
savant.drop(columns=['Name'], inplace=True)


In [6]:
# Convert baseball_reference_names to a set for efficient lookup
baseball_reference_name_set = set(baseball_reference_names)

# Filter out names from savant and fangraph dataframes that are in baseball_reference_name_set
filtered_savant = savant[~savant['player_name'].isin(baseball_reference_name_set)]
filtered_fangraph = fangraph[~fangraph['Name'].isin(baseball_reference_name_set)]

# Now, filtered_savant and filtered_fangraph contain rows where names are not in baseball_reference_names


In [7]:
savant = filtered_savant
fangraph = filtered_fangraph

In [8]:
# examine pitcher and batter and des, ', des'
temp = savant[['pitcher', 'home_team', 'away_team', 'player_name', 'batter', 'home_score', 'away_score', 'description', 'type', 'events', 'balls', 'strikes', 'game_date', 'game_year', 'game_pk', 'game_type', 'inning', 'outs_when_up', 'role_key', 'rp_indicator', 'sp_indicator', 'times_faced',  'inning_topbot', 'outs_when_up']]
#temp_merge = pd.merge(temp, temp_fangraphs, on='pitcher', how='left')

### 2. Sort the DataFrame by game date, game, then by inning, and create a top and bottom of the inning DataFrame

In [9]:
def sort_baseball_innings(df):
    """
    Sort the DataFrame by game date, game, then by inning, and within each inning,
    ensure 'Top' comes before 'Bot'. Handles duplicate column names.

    Args:
    df (pd.DataFrame): The DataFrame containing baseball game data.

    Returns:
    pd.DataFrame: The sorted DataFrame.
    """
    # Check for duplicate column names
    if df.columns.duplicated().any():
        df = df.loc[:, ~df.columns.duplicated()]

    # Define a custom sort order for 'inning_topbot'
    inning_topbot_order = {'Top': 0, 'Bot': 1}
    
    # Apply the custom sort order to create a new column for sorting
    df['inning_topbot_sort'] = df['inning_topbot'].map(inning_topbot_order)
    
    # Sort by 'game_date', 'game_pk', 'inning', 'inning_topbot_sort', then 'outs_when_up'
    df_sorted = df.sort_values(by=['game_date', 'game_pk', 'inning', 'inning_topbot_sort', 'outs_when_up'],
                               ascending=[True, True, True, True, True])
    
    # Drop the temporary sort column
    df_sorted = df_sorted.drop(columns=['inning_topbot_sort'])
    
    return df_sorted

temp_sorted = sort_baseball_innings(temp)


## 3. Create a new unique at bat identifier for each at bat and feature engineer bad walks: 2 out walks, 0-2/1-2 walks, and leadoff walks

In [10]:
def create_unique_at_bat_id(df):
    """
    Adds a unique ID for each at-bat to the DataFrame. A new at-bat is indicated by a change in the 'batter' column value.
    The unique ID will increment each time a new at-bat starts.
    """
    # Detect changes in the 'batter' column to indicate a new at-bat
    df['new_at_bat'] = df['batter'] != df['batter'].shift(1)

    # Fill the initial row's NaN value after shifting with False
    df['new_at_bat'].iloc[0] = False

    # Create a unique at-bat ID that increments with each new at-bat
    df['at_bat_id'] = df['new_at_bat'].cumsum()

    # Drop the 'new_at_bat' helper column as it's no longer needed
    df.drop(columns=['new_at_bat'], inplace=True)

    return df

def feature_engineer_walks(df):
    """
    Adds binary features for 2-out walks, 0-2/1-2 walks, and leadoff walks.

    Args:
    df (pd.DataFrame): DataFrame containing pitch-by-pitch baseball data with a rolling count.

    Returns:
    pd.DataFrame: DataFrame with the added binary features.
    """
    # Identify the last pitch of each at-bat
    df['at_bat_end'] = df['balls'].shift(-1) == 0

    # Check for walks with 2 outs
    df['2_out_walk'] = ((df['outs_when_up'] == 2) & (df['events'] == 'walk') & df['at_bat_end']).astype(int)

    # Check for walks with 0-2 or 1-2 counts
    df['0_2_or_1_2_walk'] = ((df['strikes'] == 2) & (df['balls'] < 2) & (df['events'] == 'walk') & df['at_bat_end']).astype(int)

    # Identify leadoff walks by checking if the walk is the first event in an inning
    df['inning_leadoff'] = df.groupby(['game_pk', 'inning'])['at_bat_id'].transform('min')
    df['leadoff_walk'] = (df['at_bat_id'] == df['inning_leadoff']) & (df['events'] == 'walk').astype(int)

    # Drop the helper columns
    df.drop(columns=['at_bat_end', 'inning_leadoff'], inplace=True)

    return df

# Assuming 'temp_sorted' is your DataFrame that includes the necessary columns
# Apply the function to the dataframe
temp_sorted = create_unique_at_bat_id(temp_sorted)  # Ensure each at-bat has a unique ID
temp_sorted = feature_engineer_walks(temp_sorted)

# Display the DataFrame to verify the new


C:\Users\Julian Sotelo\AppData\Local\Temp\ipykernel_28992\3014452984.py:10: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['new_at_bat'].iloc[0] = False
C:\Users\Julian Sotelo\AppData\Local\Temp\ipykernel_28992\3014452984.py:10: SettingWit

## 4. Average Outing Metrics: Batters Faced per appearance and average innings pitched per appearance

In [11]:
def calculate_average_outing_metrics(df):
    """
    Calculates the average number of batters faced and innings pitched per appearance for each pitcher
    based on their role and season, with separate calculations for starting pitchers and relief pitchers.

    Args:
    df (pd.DataFrame): DataFrame containing baseball game data with pitcher IDs, role, season, and game_pk.

    Returns:
    pd.DataFrame: DataFrame with each pitcher's average batters faced per appearance and innings pitched per appearance,
                  with separate columns for starting pitchers and relief pitchers.
    """
    # Initialize DataFrames to hold average metrics for SP and RP
    metrics_columns = ['average_batters_faced_per_appearance_SP', 'average_innings_pitched_per_appearance_SP',
                       'average_batters_faced_per_appearance_RP', 'average_innings_pitched_per_appearance_RP']
    average_outing_metrics = pd.DataFrame(columns=['pitcher', 'game_year'] + metrics_columns)

    for role_key in ['SP', 'RP']:
        # Group by 'pitcher', 'role_key', 'game_year', and 'game_pk' to count distinct batters faced per appearance
        batters_faced_per_appearance = df[df['role_key'] == role_key].groupby(['pitcher', 'game_year', 'game_pk'])['batter'].nunique().reset_index(name='batters_faced')

        # Group by 'pitcher', 'role_key', 'game_year', 'game_pk', and 'inning_topbot' to count distinct innings pitched per appearance
        innings_pitched_per_appearance = df[df['role_key'] == role_key].groupby(['pitcher', 'game_year', 'game_pk', 'inning_topbot'])['inning'].nunique().groupby(['pitcher', 'game_year', 'game_pk']).sum().reset_index(name='innings_pitched')

        # Calculate the average batters faced per appearance for each pitcher by role and season
        average_batters_faced = batters_faced_per_appearance.groupby(['pitcher', 'game_year'])['batters_faced'].mean().reset_index(name=f'average_batters_faced_per_appearance_{role_key}')

        # Calculate the average innings pitched per appearance for each pitcher by role and season
        average_innings_pitched = innings_pitched_per_appearance.groupby(['pitcher', 'game_year'])['innings_pitched'].mean().reset_index(name=f'average_innings_pitched_per_appearance_{role_key}')

        # Merge the two metrics into the average_outing_metrics DataFrame
        role_metrics = pd.merge(average_batters_faced, average_innings_pitched, on=['pitcher', 'game_year'])
        average_outing_metrics = average_outing_metrics.merge(role_metrics, on=['pitcher', 'game_year'], how='outer')

    return average_outing_metrics.fillna(0)

# Assuming temp_sorted is your DataFrame that includes the necessary columns
# Calculate the average outing metrics
average_outing_metrics = calculate_average_outing_metrics(temp_sorted)

# Merge the calculated metrics back into the temp_sorted DataFrame
temp_sorted = temp_sorted.merge(average_outing_metrics, on=['pitcher', 'game_year'], how='left')

C:\Users\Julian Sotelo\AppData\Local\Temp\ipykernel_28992\3417247848.py:35: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return average_outing_metrics.fillna(0)


## 5. Average Inning a  pitcher pitches

In [13]:
def average_inning_per_pitcher(df):
    """
    Determines the average inning in which each pitcher typically pitches by role and season, with separate calculations for starting pitchers and relief pitchers.

    Args:
    df (pd.DataFrame): DataFrame containing baseball game data with pitcher IDs, role, season, and game_pk.

    Returns:
    pd.DataFrame: DataFrame with each pitcher, their role, season, and their average pitched inning for each role.
    """
    # Initialize an empty DataFrame to store results
    average_innings_df = pd.DataFrame()

    for role_key in ['SP', 'RP']:
        # Group by pitcher, role, season, and calculate the mean inning
        role_average_innings = df[df['role_key'] == role_key].groupby(['pitcher', 'game_year'])['inning'].mean().reset_index(name=f'average_inning_{role_key}')

        # Merge results into the main DataFrame
        if average_innings_df.empty:
            average_innings_df = role_average_innings
        else:
            average_innings_df = pd.merge(average_innings_df, role_average_innings, on=['pitcher', 'game_year'], how='outer')

    return average_innings_df.fillna('NA')

# Calculate the average inning per pitcher considering their role and season
average_inning_per_pitcher_df = average_inning_per_pitcher(temp_sorted)

# Merge the average inning per pitcher, role, and season back into the temp_sorted dataframe
temp_sorted = temp_sorted.merge(average_inning_per_pitcher_df, on=['pitcher', 'game_year'], how='left')


## 6. Average Inning Entry Exit

In [14]:
def calculate_pitcher_inning_entry_exit(df):
    """
    Calculates the average inning entry and exit for each pitcher by role and season based on pitch-by-pitch data,
    with separate calculations for starting pitchers and relief pitchers.

    Args:
    df (pd.DataFrame): DataFrame containing pitch-by-pitch baseball game data.

    Returns:
    pd.DataFrame: DataFrame with each pitcher's average inning of entry and exit by role and season.
    """
    # Initialize an empty DataFrame to store results
    pitcher_inning_stats_df = pd.DataFrame()

    for role_key in ['SP', 'RP']:
        # Identify the rows where a new pitcher starts or an existing pitcher ends their appearance for the specific role
        role_df = df[df['role_key'] == role_key]
        role_df['pitcher_start'] = role_df['pitcher'] != role_df['pitcher'].shift()
        role_df['pitcher_end'] = role_df['pitcher'] != role_df['pitcher'].shift(-1)

        # Filter the dataset to get only the rows where pitchers start and end their appearance
        start_innings = role_df[role_df['pitcher_start']].groupby(['pitcher', 'game_year', 'game_pk'])['inning'].first().reset_index()
        end_innings = role_df[role_df['pitcher_end']].groupby(['pitcher', 'game_year', 'game_pk'])['inning'].last().reset_index()

        # Calculate the average inning of entry and exit for each pitcher by role and season
        avg_start_innings = start_innings.groupby(['pitcher', 'game_year'])['inning'].mean().reset_index(name=f'avg_inning_entry_{role_key}')
        avg_end_innings = end_innings.groupby(['pitcher', 'game_year'])['inning'].mean().reset_index(name=f'avg_inning_exit_{role_key}')

        # Merge role-specific stats into the main DataFrame
        role_metrics = pd.merge(avg_start_innings, avg_end_innings, on=['pitcher', 'game_year'])
        if pitcher_inning_stats_df.empty:
            pitcher_inning_stats_df = role_metrics
        else:
            pitcher_inning_stats_df = pd.merge(pitcher_inning_stats_df, role_metrics, on=['pitcher', 'game_year'], how='outer')

    return pitcher_inning_stats_df.fillna('NA')

# Calculate the pitcher inning stats considering the pitcher's role and season
pitcher_inning_stats = calculate_pitcher_inning_entry_exit(temp_sorted)

# Merge the stats back into the original DataFrame with a consideration of the role and season
temp_sorted = temp_sorted.merge(pitcher_inning_stats, on=['pitcher', 'game_year'], how='left')

# Display the result
print(temp_sorted[['pitcher', 'game_year', 'avg_inning_entry_SP', 'avg_inning_exit_SP', 'avg_inning_entry_RP', 'avg_inning_exit_RP']].drop_duplicates().head())


C:\Users\Julian Sotelo\AppData\Local\Temp\ipykernel_28992\706176459.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  role_df['pitcher_start'] = role_df['pitcher'] != role_df['pitcher'].shift()
C:\Users\Julian Sotelo\AppData\Local\Temp\ipykernel_28992\706176459.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  role_df['pitcher_end'] = role_df['pitcher'] != role_df['pitcher'].shift(-1)
C:\Users\Julian Sotelo\AppData\Local\Temp\ipykernel_28992\706176459.py:18: SettingWithCopyWarning: 
A value is trying 

     pitcher  game_year avg_inning_entry_SP avg_inning_exit_SP  \
0     608566       2021                 1.0            5.78125   
23    477132       2021                 1.0           5.636364   
136   592346       2021                 1.0                5.0   
211   608349       2021                 1.0                1.5   
215   622075       2021                  NA                 NA   

    avg_inning_entry_RP avg_inning_exit_RP  
0                    NA                 NA  
23                   NA                 NA  
136            5.333333                7.0  
211            7.565217           7.695652  
215              6.9375             7.1875  


## 7. Mode Inning Entry and Exit

In [15]:
import pandas as pd
from scipy.stats import mode

def calculate_pitcher_inning_entry_exit(df):
    """
    Calculates the mode inning entry and exit for each pitcher by role and season based on pitch-by-pitch data, 
    with separate calculations for starting pitchers and relief pitchers.

    Args:
    df (pd.DataFrame): DataFrame containing pitch-by-pitch baseball game data.

    Returns:
    pd.DataFrame: DataFrame with each pitcher's mode inning of entry and exit by role and season.
    """
    # Initialize an empty DataFrame to store results
    pitcher_inning_stats_df = pd.DataFrame()

    for role_key in ['SP', 'RP']:
        # Identify the rows where a new pitcher starts or an existing pitcher ends their appearance
        role_df = df[df['role_key'] == role_key].copy()
        role_df['pitcher_start'] = role_df['pitcher'] != role_df['pitcher'].shift()
        role_df['pitcher_end'] = role_df['pitcher'] != role_df['pitcher'].shift(-1)

        # Filter the dataset to get only the rows where pitchers start and end their appearance
        start_innings = role_df[role_df['pitcher_start']].groupby(['pitcher', 'game_year', 'game_pk'])['inning'].first().reset_index()
        end_innings = role_df[role_df['pitcher_end']].groupby(['pitcher', 'game_year', 'game_pk'])['inning'].last().reset_index()

        # Custom function to calculate mode
        def calculate_mode(series):
            if series.empty:
                return None
            return mode(series)[0][0]

        # Calculate the mode inning of entry and exit for each pitcher by role and season
        mode_start_innings = start_innings.groupby(['pitcher', 'game_year'])['inning'].apply(calculate_mode).reset_index(name=f'mode_inning_entry_{role_key}')
        mode_end_innings = end_innings.groupby(['pitcher', 'game_year'])['inning'].apply(calculate_mode).reset_index(name=f'mode_inning_exit_{role_key}')

        # Merge results into the main DataFrame
        role_metrics = pd.merge(mode_start_innings, mode_end_innings, on=['pitcher', 'game_year'])
        if pitcher_inning_stats_df.empty:
            pitcher_inning_stats_df = role_metrics
        else:
            pitcher_inning_stats_df = pd.merge(pitcher_inning_stats_df, role_metrics, on=['pitcher', 'game_year'], how='outer')

    return pitcher_inning_stats_df.fillna('NA')
"""
# Calculate the pitcher inning stats considering the pitcher's role and season
pitcher_inning_stats = calculate_pitcher_inning_entry_exit(temp_sorted)

# Merge the stats back into the original DataFrame with a consideration of the role and season
temp_sorted = temp_sorted.merge(pitcher_inning_stats, on=['pitcher', 'game_year'], how='left')"""


"\n# Calculate the pitcher inning stats considering the pitcher's role and season\npitcher_inning_stats = calculate_pitcher_inning_entry_exit(temp_sorted)\n\n# Merge the stats back into the original DataFrame with a consideration of the role and season\ntemp_sorted = temp_sorted.merge(pitcher_inning_stats, on=['pitcher', 'game_year'], how='left')"

## 8. Calculate Walk Rates per Year

In [16]:
def calculate_walk_rates(df):
    """
    Calculates the rates of "bad walks" and "leadoff walks" for each pitcher by role and season,
    with separate calculations for starting pitchers and relief pitchers.

    Args:
    df (pd.DataFrame): DataFrame containing pitch-by-pitch baseball game data.

    Returns:
    pd.DataFrame: DataFrame with each pitcher's "bad walk" rate and "leadoff walk" rate by role and season.
    """
    # Initialize an empty DataFrame to store results
    walk_rates_df = pd.DataFrame()

    for role_key in ['SP', 'RP']:
        # Calculate the number of "bad walks" for each pitcher in the given role
        role_df = df[df['role_key'] == role_key]
        role_df['bad_walk'] = role_df['2_out_walk'] | role_df['0_2_or_1_2_walk']
        bad_walks_per_pitcher = role_df.groupby(['pitcher', 'game_year'])['bad_walk'].sum()

        # Calculate the number of leadoff walks for each pitcher in the given role
        leadoff_walks_per_pitcher = role_df.groupby(['pitcher', 'game_year'])['leadoff_walk'].sum()

        # Calculate the total number of batters faced by each pitcher in the given role
        batters_faced_per_pitcher = role_df.groupby(['pitcher', 'game_year'])['at_bat_id'].nunique()

        # Calculate the rate of bad walks and leadoff walks by role and season
        bad_walk_rate_per_pitcher = bad_walks_per_pitcher / batters_faced_per_pitcher
        leadoff_walk_rate_per_pitcher = leadoff_walks_per_pitcher / batters_faced_per_pitcher

        # Create a DataFrame for the role-specific walk rates
        role_walk_rates = pd.DataFrame({
            'pitcher': bad_walk_rate_per_pitcher.index.get_level_values('pitcher'),
            'game_year': bad_walk_rate_per_pitcher.index.get_level_values('game_year'),
            f'bad_walk_rate_{role_key}': bad_walk_rate_per_pitcher.values,
            f'leadoff_walk_rate_{role_key}': leadoff_walk_rate_per_pitcher.values
        })

        # Merge role-specific walk rates into the main DataFrame
        if walk_rates_df.empty:
            walk_rates_df = role_walk_rates
        else:
            walk_rates_df = pd.merge(walk_rates_df, role_walk_rates, on=['pitcher', 'game_year'], how='outer')

    return walk_rates_df.fillna('NA')

# Calculate the walk rates considering the pitcher's role and season
walk_rates = calculate_walk_rates(temp_sorted)

# Merge the walk rates back into the original DataFrame with a consideration of the role and season
temp_sorted = temp_sorted.merge(walk_rates, on=['pitcher', 'game_year'], how='left')

C:\Users\Julian Sotelo\AppData\Local\Temp\ipykernel_28992\3344310496.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  role_df['bad_walk'] = role_df['2_out_walk'] | role_df['0_2_or_1_2_walk']
C:\Users\Julian Sotelo\AppData\Local\Temp\ipykernel_28992\3344310496.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  role_df['bad_walk'] = role_df['2_out_walk'] | role_df['0_2_or_1_2_walk']


## 9. Find the most common inning a pitcher pitches in and create a new feature for that

In [17]:
def most_common_inning_per_pitcher(df):
    """
    Calculates the most common inning that each pitcher pitches by role and season, with separate columns for starting pitchers (SP) and relief pitchers (RP).

    Args:
    df (pd.DataFrame): DataFrame containing baseball game data.

    Returns:
    pd.DataFrame: DataFrame with each pitcher's most common inning by role and season.
    """
    # Initialize an empty DataFrame to store results
    most_common_innings_df = pd.DataFrame()

    for role_key in ['SP', 'RP']:
        # Group by pitcher, role, season, and inning, then count the number of appearances in each inning
        inning_counts = df[df['role_key'] == role_key].groupby(['pitcher', 'game_year', 'inning']).size().reset_index(name='counts')

        # Find the inning with the maximum count (mode) for each pitcher by role and season
        most_common_innings = inning_counts.loc[inning_counts.groupby(['pitcher', 'game_year'])['counts'].idxmax()]

        # Rename columns for clarity
        most_common_innings.rename(columns={'inning': f'most_common_inning_{role_key}'}, inplace=True)
        most_common_innings.drop('counts', axis=1, inplace=True)

        # Merge results into the main DataFrame
        if most_common_innings_df.empty:
            most_common_innings_df = most_common_innings
        else:
            most_common_innings_df = pd.merge(most_common_innings_df, most_common_innings, on=['pitcher', 'game_year'], how='outer')

    return most_common_innings_df.fillna('NA')

# Calculate the most common inning per pitcher considering their role and season
most_common_inning_df = most_common_inning_per_pitcher(temp_sorted)

# Merge the most common inning data back into the original DataFrame
temp_sorted = temp_sorted.merge(most_common_inning_df, on=['pitcher', 'game_year'], how='left')

##### Ninth Inning Pitchers

In [19]:
# FInd pitchers with the most common inning in the 9th inning
ninth_inning_pitchers = temp_sorted[temp_sorted['most_common_inning_RP'] == 9]
ninth_inning_pitchers = ninth_inning_pitchers[['pitcher', 'player_name', 'most_common_inning_RP', 'game_year']]
ninth_inning_pitchers = ninth_inning_pitchers.drop_duplicates()
ninth_inning_pitchers.shape

(291, 4)

##### Eighth Inning Pitchers

In [20]:
# FInd pitchers with the most common inning in the 8th inning
eighth_inning_pitchers = temp_sorted[temp_sorted['most_common_inning_RP'] == 8]
eighth_inning_pitchers = eighth_inning_pitchers[['pitcher', 'player_name', 'most_common_inning_RP', 'game_year']]
eighth_inning_pitchers = eighth_inning_pitchers.drop_duplicates()
eighth_inning_pitchers.head()


,pitcher,player_name,most_common_inning_RP,game_year
211,608349,Corey Knebel,8.0,2021
215,622075,Yency Almonte,8.0,2021
521,656464,Kevin Ginkel,8.0,2021
564,641941,Emilio Pagan,8.0,2021
604,519141,Drew Pomeranz,8.0,2021


##### Seventh Inning or Later

In [21]:
# FInd pitchers with the most common inning in the 7th inning   
seventh_inning_pitchers = temp_sorted[(temp_sorted['most_common_inning_RP'] == 7) & (temp_sorted['role_key'] == 'RP')]
seventh_inning_pitchers = seventh_inning_pitchers[['pitcher', 'player_name', 'most_common_inning_RP', 'game_year']]
seventh_inning_pitchers = seventh_inning_pitchers.drop_duplicates()
seventh_inning_pitchers.shape

(550, 4)

##### Sixth Inning Relief Pitchers

In [22]:
# FInd pitchers with the most common inning in the 6th inning   
sixth_inning_pitchers = temp_sorted[(temp_sorted['most_common_inning_RP'] == 6) & (temp_sorted['role_key'] == 'RP')]
sixth_inning_pitchers = sixth_inning_pitchers[['pitcher', 'player_name', 'most_common_inning_RP', 'game_year']]
sixth_inning_pitchers = sixth_inning_pitchers.drop_duplicates()
sixth_inning_pitchers.shape


(301, 4)

##### Fifth Inning Relief

In [23]:
# FInd pitchers with the most common inning in the 5th inning   
fifth_inning_pitchers = temp_sorted[(temp_sorted['most_common_inning_RP'] == 5) & (temp_sorted['role_key'] == 'RP')]
fifth_inning_pitchers = fifth_inning_pitchers[['pitcher', 'role_key', 'player_name', 'most_common_inning_RP', 'game_year']]
fifth_inning_pitchers = fifth_inning_pitchers.drop_duplicates()
fifth_inning_pitchers.shape

(107, 5)

##### Fourth Inning Relief Pitchers

In [24]:
# fourth inning relief pitchers 
fourth_inning_pitchers = temp_sorted[(temp_sorted['most_common_inning_RP'] == 4) & (temp_sorted['role_key'] == 'RP')]
fourth_inning_pitchers = fourth_inning_pitchers[['pitcher', 'role_key', 'player_name', 'most_common_inning_RP', 'game_year']]
fourth_inning_pitchers = fourth_inning_pitchers.drop_duplicates()
fourth_inning_pitchers.shape

(63, 5)

In [25]:
# sample of the fourth inning relief pitchers
fourth_inning_pitchers.sample(5)

,pitcher,role_key,player_name,most_common_inning_RP,game_year
121572,624586,RP,James Hoyt,4.0,2021
1498119,683769,RP,Hunter Gaddis,4.0,2023
1693414,666214,RP,Joey Wentz,4.0,2023
869213,642232,RP,Ryan Yarbrough,4.0,2022
62730,607229,RP,Robert Gsellman,4.0,2021


In [26]:
# find pitchers who pitched the most in the 4th inning
sean_reid_foley = temp_sorted[(temp_sorted['pitcher'] == 656887) & (temp_sorted['role_key'] == 'RP')]
cristopher_sanchez = temp_sorted[(temp_sorted['pitcher'] == 650911) & (temp_sorted['role_key'] == 'RP')]
levi_stoudt = temp_sorted[(temp_sorted['pitcher'] == 686651) & (temp_sorted['role_key'] == 'RP')]
mitch_keller = temp_sorted[(temp_sorted['pitcher'] == 656605) & (temp_sorted['role_key'] == 'RP')]
josh_flemming = temp_sorted[(temp_sorted['pitcher'] == 676596) & (temp_sorted['role_key'] == 'RP')]

##### Third Inning Relief Pitching

In [27]:
# third inning relief pitchers
third_inning_pitchers = temp_sorted[(temp_sorted['most_common_inning_RP'] == 3) & (temp_sorted['role_key'] == 'RP')]
third_inning_pitchers = third_inning_pitchers[['pitcher', 'role_key', 'player_name', 'most_common_inning_RP', 'game_year']]
third_inning_pitchers = third_inning_pitchers.drop_duplicates()
third_inning_pitchers.shape

(24, 5)

In [28]:
# find pitchers who pitched in the third inning
third_inning_pitchers.sample(5)

,pitcher,role_key,player_name,most_common_inning_RP,game_year
1234076,674681,RP,Michael Plassmeyer,3.0,2022
770871,675921,RP,Spencer Howard,3.0,2022
666531,664337,RP,Miguel Yajure,3.0,2021
688543,456501,RP,Johnny Cueto,3.0,2021
170853,669145,RP,Bruce Zimmermann,3.0,2021


In [29]:
michael_plassmeyer = temp_sorted[(temp_sorted['pitcher'] == 674681	) & (temp_sorted['role_key'] == 'RP')]
carson_fulmer = temp_sorted[(temp_sorted['pitcher'] == 608334) & (temp_sorted['role_key'] == 'RP')]
adrian_sampson = temp_sorted[(temp_sorted['pitcher'] == 592716) & (temp_sorted['role_key'] == 'RP')]
drew_smyly = temp_sorted[(temp_sorted['pitcher'] == 592767) & (temp_sorted['role_key'] == 'RP')]
daniel_castano = temp_sorted[(temp_sorted['pitcher'] == 641447) & (temp_sorted['role_key'] == 'RP')]

##### Second Inning Relief Pitching

In [30]:
# second inning relief pitchers
second_inning_pitchers = temp_sorted[(temp_sorted['most_common_inning_RP'] == 2) & (temp_sorted['role_key'] == 'RP')]
second_inning_pitchers = second_inning_pitchers[['pitcher', 'role_key', 'player_name', 'most_common_inning_RP', 'game_year']]
second_inning_pitchers = second_inning_pitchers.drop_duplicates()
second_inning_pitchers

,pitcher,role_key,player_name,most_common_inning_RP,game_year
30169,643615,RP,T.J. Zeuch,2.0,2021
490215,448179,RP,Rich Hill,2.0,2021
668960,641712,RP,Dakota Hudson,2.0,2021
1305682,640455,RP,Sean Manaea,2.0,2022
1379063,571760,RP,Andrew Heaney,2.0,2022
1906734,622663,RP,Luis Severino,2.0,2023
2046545,682990,RP,Quinn Priester,2.0,2023


In [31]:
# Find games that T.J. Zeuch pitched in but did not start
tj_zeuch = temp_sorted[(temp_sorted['pitcher'] == 643615) & (temp_sorted['role_key'] == 'RP')]
rich_hill = temp_sorted[(temp_sorted['pitcher'] == 448179) & (temp_sorted['role_key'] == 'RP')]
dakota_hudson = temp_sorted[(temp_sorted['pitcher'] == 641712) & (temp_sorted['role_key'] == 'RP')]
sean_manaea = temp_sorted[(temp_sorted['pitcher'] == 640455) & (temp_sorted['role_key'] == 'RP')]
andrea_heaney = temp_sorted[(temp_sorted['pitcher'] == 571760) & (temp_sorted['role_key'] == 'RP')]
luis_severino = temp_sorted[(temp_sorted['pitcher'] == 622663) & (temp_sorted['role_key'] == 'RP')]
quinn_priester = temp_sorted[(temp_sorted['pitcher'] == 682990) & (temp_sorted['role_key'] == 'RP')]

## Determine the Role for Relief Pitchers (Median)

In [32]:
def classify_relief_pitchers(df):
    """
    Classify relief pitchers based on their most common inning.

    Args:
    df (pd.DataFrame): DataFrame containing baseball game data with columns for most common inning.

    Returns:
    pd.Series: Series specifying the relief pitcher's role for each entry.
    """
    # Function to determine the relief role
    def get_relief_role(inning):
        if inning in [2, 3, 4]:
            return 'Long Reliever'
        elif inning in [5, 6]:
            return 'Middle Reliever'
        elif inning in [7, 8]:
            return 'Setup Pitcher'
        elif inning == 9:
            return 'Closer'
        else:
            return 'Starting Pitcher'

    # Apply the function to each row in the DataFrame
    return df['most_common_inning_RP'].apply(get_relief_role)

# Assign the result to a new column in temp_sorted
temp_sorted['classified_role'] = classify_relief_pitchers(temp_sorted)


In [34]:
def classify_starting_pitchers(df):
    # Convert 'average_inning_SP' to boolean
    return df['average_inning_SP'].astype(bool)

def classify_middle_relief_pitchers(df):
    # Convert 'average_inning_RP' to numeric and check the range for middle relief
    avg_inning_rp = pd.to_numeric(df['average_inning_RP'], errors='coerce')
    return (df['role_key'] == 'RP') & avg_inning_rp.between(5, 7)

def classify_setup_pitchers(df):
    # Convert 'average_inning_RP' to numeric and check the range for setup pitchers
    avg_inning_rp = pd.to_numeric(df['average_inning_RP'], errors='coerce')
    return (df['role_key'] == 'RP') & avg_inning_rp.between(7, 8)

def classify_closers(df):
    # Convert 'average_inning_RP' to numeric and check for closers
    avg_inning_rp = pd.to_numeric(df['average_inning_RP'], errors='coerce')
    return (df['role_key'] == 'RP') & (avg_inning_rp >= 9)

def classify_long_relief_pitchers(df):
    # Convert 'average_innings_pitched_per_appearance_RP' to numeric and check for long relievers
    avg_innings = pd.to_numeric(df['average_innings_pitched_per_appearance_RP_x'], errors='coerce')
    return (df['role_key'] == 'RP') & (avg_innings > avg_innings.median())

def classify_pitchers(df):
    df['pitcher_role'] = 'Unknown'  # Default classification

    df.loc[classify_starting_pitchers(df), 'pitcher_role'] = 'Starting Pitcher'
    df.loc[classify_middle_relief_pitchers(df), 'pitcher_role'] = 'Middle Relief Pitcher'
    df.loc[classify_setup_pitchers(df), 'pitcher_role'] = 'Setup Pitcher'
    df.loc[classify_closers(df), 'pitcher_role'] = 'Closer'
    df.loc[classify_long_relief_pitchers(df), 'pitcher_role'] = 'Long Reliever'

    return df

# Assuming temp_sorted is your DataFrame that includes the necessary columns
#temp_sorted = classify_pitchers(temp_sorted)


In [36]:
# remove all post fixes of _x and _y from column names
temp_sorted.columns = temp_sorted.columns.str.replace('_x', '')
#temp_sorted.columns = temp_sorted.columns.str.replace('_y', '')

# drop all identical columns
temp_sorted = temp_sorted.loc[:,~temp_sorted.columns.duplicated()]

# return all columns are filled with 100% 0s
temp_sorted.isin([0]).all()

# drop all columns that are filled with 100% 0s
temp_sorted = temp_sorted.loc[:, (temp_sorted != 0).any(axis=0)]
temp_sorted.columns = temp_sorted.columns.str.replace('_y', '')
temp_sorted = temp_sorted.loc[:,~temp_sorted.columns.duplicated()]

# replace gamear with game_year

temp_sorted.rename(columns={'gameear': 'game_year'}, inplace=True)

## Save the DataFrame

In [37]:
# Assuming temp_sorted and fangraph are your original DataFrames
preferred_data = ['pitcher', 'player_name', 'most_common_inning_SP',
       'most_common_inning_RP', 'average_batters_faced_per_appearance_SP',
       'average_innings_pitched_per_appearance_SP',
       'average_batters_faced_per_appearance_RP',
       'average_innings_pitched_per_appearance_RP', 'average_inning_SP',
       'average_inning_RP', 'avg_inning_entry_SP', 'avg_inning_exit_SP',
       'avg_inning_entry_RP', 'avg_inning_exit_RP', 'bad_walk_rate_SP',
       'leadoff_walk_rate_SP', 'bad_walk_rate_RP', 'leadoff_walk_rate_RP', 'classified_role']
# pitcher role removed from the list

temp_sorted_test = temp_sorted[preferred_data]

# drop the duplicates from the pitcher column
temp_sorted_test.drop_duplicates(subset=['pitcher'], inplace=True)

C:\Users\Julian Sotelo\AppData\Local\Temp\ipykernel_28992\3367328474.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_sorted_test.drop_duplicates(subset=['pitcher'], inplace=True)


In [38]:
from src.paths import CLEANED_DATA_DIR

# rename the ID column to match the savant data
fangraph = fangraph.rename(columns={'MLBAMID': 'pitcher'})

# merge the fangraph data with the temp_sorted_test data
fangraph_merged = pd.merge(fangraph, temp_sorted_test, on='pitcher', how='left')

# save the fangraph_merged data to a csv file
fangraph_merged.to_csv(CLEANED_DATA_DIR / 'fangraphs_engineered_merged.csv', index=False)